In [19]:
import sys
import argparse
import numpy as np
import cv2
import datetime
import time
import pytz
import json


In [20]:
[t for t in pytz.all_timezones if t.startswith('America/Mex')]

['America/Mexico_City']

In [21]:
import sys
import numpy as np
import cv2
import pdfplumber

In [10]:
def load_whole_pdf(pdf_path):

    # initialization
    image_list = []

    # read PDF file (load all pages in the PDF file)
    name = pdf_path.lower()
    if name.endswith('.pdf'):
        with pdfplumber.open(pdf_path) as pdf:
            page_count = len(pdf.pages)
            for page_index in range(page_count):  # traverse all pages
                page = pdf.pages[page_index]  # select the current page
                page_image = page.to_image(resolution=150) # convert the page to image by default (20230815)
                image = cv2.cvtColor(np.array(page_image.original), cv2.COLOR_RGB2BGR)

                image_list.append(image)

            pdf.close()

    return image_list

In [22]:
load_whole_pdf

<function __main__.load_whole_pdf(pdf_path)>

In [23]:
from langchain_core.documents import Document
#from pymupdf4llm import to_markdown
from pathlib import Path
import os
import time
import logging
#from find_info_app.l

In [24]:
root_string = '/home/camilo/Documents/by_species/' #Conabio
#root_string = '/home/camilo/Documents/00-Conabio/by_species/' #Casa
root_folder = Path(root_string)
#This can be a file in the root dir to read from
processed_list=['Leptonycteris yerbabuenae', 'Leptonycteris nivalis', 'Melipona beecheii'] # Check dedazo en by_species de Conabio 'Leptonycteris yerbabuenae'
species_subfolders = os.listdir(root_folder)
pending_process_species = list(set(species_subfolders)-set(processed_list))
pending_process_species 

['test species_A', 'test species2']

In [26]:
def load_whole_pdf(pdf_path):

    # initialization
    image_list = []

    # read PDF file (load all pages in the PDF file)
    name = pdf_path.lower()
    if name.endswith('.pdf'):
        with pdfplumber.open(pdf_path) as pdf:
            page_count = len(pdf.pages)
            for page_index in range(page_count):  # traverse all pages
                page = pdf.pages[page_index]  # select the current page
                page_image = page.to_image(resolution=150) # convert the page to image by default (20230815)
                image = cv2.cvtColor(np.array(page_image.original), cv2.COLOR_RGB2BGR)

                image_list.append(image)

            pdf.close()

    return image_list

In [28]:
import pdfplumber

my_full_path = '/home/camilo/Documents/00-Conabio/by_species/test species_A/test species_A_bibliografía/Arita, 1991.pdf' #casa
my_full_path = '/home/camilo/Documents/by_species/test species_A/test species_A_bibliografía/Arita, 1991.pdf' #Conabio
my_full_path


'/home/camilo/Documents/by_species/test species_A/test species_A_bibliografía/Arita, 1991.pdf'

In [31]:
image_list = []

with pdfplumber.open(my_full_path) as pdf:
    page_count = len(pdf.pages)
    for page_index in range(page_count):  # traverse all pages
        page = pdf.pages[page_index]  # select the current page
        page_image = page.to_image(resolution=150) # convert the page to image by default (20230815)
        image = cv2.cvtColor(np.array(page_image.original), cv2.COLOR_RGB2BGR)

        image_list.append(image)

    pdf.close()

In [32]:
len(image_list)

10

In [33]:
from pipelines.document_structurization import DocumentStructurization

def whole_pdf_conversion_example(image_list):

    # configure
    configs = dict()
    
    layout_analysis_configs = dict()
    layout_analysis_configs['from_modelscope_flag'] = False
    layout_analysis_configs['model_path'] = '/home/DocXLayout_231012.pth'  # note that: currently the layout analysis model is NOT from modelscope
    configs['layout_analysis_configs'] = layout_analysis_configs
    
    text_detection_configs = dict()
    text_detection_configs['from_modelscope_flag'] = True
    text_detection_configs['model_path'] = 'damo/cv_resnet18_ocr-detection-line-level_damo'
    configs['text_detection_configs'] = text_detection_configs

    text_recognition_configs = dict()
    text_recognition_configs['from_modelscope_flag'] = True
    text_recognition_configs['model_path'] = 'damo/cv_convnextTiny_ocr-recognition-document_damo'  # alternatives: 'damo/cv_convnextTiny_ocr-recognition-scene_damo', 'damo/cv_convnextTiny_ocr-recognition-general_damo', 'damo/cv_convnextTiny_ocr-recognition-handwritten_damo' 
    configs['text_recognition_configs'] = text_recognition_configs

    formula_recognition_configs = dict()
    formula_recognition_configs['from_modelscope_flag'] = False
    formula_recognition_configs['image_resizer_path'] = '/home/LaTeX-OCR_image_resizer.onnx'
    formula_recognition_configs['encoder_path'] = '/home/LaTeX-OCR_encoder.onnx'
    formula_recognition_configs['decoder_path'] = '/home/LaTeX-OCR_decoder.onnx'
    formula_recognition_configs['tokenizer_json'] = '/home/LaTeX-OCR_tokenizer.json'
    configs['formula_recognition_configs'] = formula_recognition_configs

    # initialize
    document_structurizer = DocumentStructurization(configs)

    # run
    final_result = []
    page_index = 0
    for image in image_list:
        result = document_structurizer(image)

        page_info = {'page': page_index, 'information': result}
        final_result.append(page_info)

        page_index = page_index + 1

    if True:
        print (final_result)

    # release
    document_structurizer.release()

    return final_result

/home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
final_result = whole_pdf_conversion_example(image_list)

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain/modules/../../../DocumentUnderstanding/DocXLayout/../../exp/ctdet_subfield/default
heads {'hm': 11, 'cls': 4, 'ftype': 3, 'wh': 8, 'hm_sub': 2, 'wh_sub': 8, 'reg': 2, 'reg_sub': 2}
Namespace(task='ctdet_subfield', dataset='huntie', test=False, data_src='default', exp_id='default', vis_corner=0, convert_onnx=0, onnx_path='auto', debug=0, load_model='/home/DocXLayout_231012.pth', resume=False, gpus=[0], num_workers=16, not_cuda_benchmark=False, seed=317, print_iter=0, hide_data_time=False, save_all=False, metric='loss', vis_thresh=0.3, nms_thresh=0.3, corner_thresh=0.3, debugger_theme='white', arch='dlav0subfield_34', head_conv=256, down_ratio=4, input_res=768, input_h=768, input_w=768, lr=0.000125, lr_step=[80], NotFixList='', num_epochs=90, batch_size=32, master_batch_size=32, num_iters=-1, val_intervals=5, trainval=False, negative=False, adamW

/home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain/modules/../../../DocumentUnderstanding/DocXLayout/models/model.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimenta

09/10/2024 10:33:12 - WARNING - tensorflow -   From /home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/modelscope/utils/device.py:60: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2024-09-10 10:33:12.676024: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-10 10:33:12.677201: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-09-10 10:33:12.720408: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/t

09/10/2024 10:33:15 - WARNING - tensorflow -   From /home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/modelscope/pipelines/cv/ocr_utils/ops.py:744: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


2024-09-10 10:33:15,349 - modelscope - INFO - loading model from /home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo/tf_ckpts/checkpoint-80000


09/10/2024 10:33:15 - INFO - tensorflow -   Restoring parameters from /home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo/tf_ckpts/checkpoint-80000


2024-09-10 10:33:15.772354: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2024-09-10 10:33:19,642 - modelscope - WARNING - Model revision not specified, use revision: v2.4.0
2024-09-10 10:33:20,704 - modelscope - INFO - initiate model from /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo
2024-09-10 10:33:20,706 - modelscope - INFO - initiate model from location /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo.
2024-09-10 10:33:20,709 - modelscope - INFO - initialize model from /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo
/home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/modelscope/models/cv/ocr_recognition/model.py:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to 

[{'page': 0, 'information': [{'category_index': 0, 'category_name': 'title', 'region_poly': [79, 194, 986, 194, 986, 265, 79, 265], 'text_list': [{'position': [81, 191, 986, 191, 986, 222, 81, 222], 'content': ['Spatial Segregation in Long-Nosed Bats, Leptonycteris nivalis and']}, {'position': [82, 230, 547, 231, 547, 261, 81, 261], 'content': ['Leptonycteris curasoae, in Mexico']}]}, {'category_index': 0, 'category_name': 'title', 'region_poly': [82, 466, 140, 466, 140, 481, 82, 481], 'text_list': [{'position': [82, 465, 137, 465, 137, 479, 82, 479], 'content': ['1 author:']}]}, {'category_index': 1, 'category_name': 'figure', 'region_poly': [92, 771, 128, 771, 128, 794, 92, 794], 'text_list': []}, {'category_index': 1, 'category_name': 'figure', 'region_poly': [92, 708, 129, 708, 129, 731, 92, 731], 'text_list': []}, {'category_index': 1, 'category_name': 'figure', 'region_poly': [82, 506, 124, 506, 124, 531, 82, 531], 'text_list': []}, {'category_index': 1, 'category_name': 'figure'

In [147]:
final_result[0]["test_info"] ="testing..."

In [152]:
final_result[1].get('page')

1

In [36]:
#list_of dictionaries
type(final_result),type(final_result[0])  

(list, dict)

In [64]:
import pandas as pd
def result_to_chunks(final_result: list, filter_criteria = ['plain text']) -> Document:
    # 1. to DataFrame and "Explode" for relevant columns
    # 2. Select type of objects, default 'plain text' category
    # 3. Group text into region-polygon-chunks

    # 1
    df = pd.DataFrame(final_result)
    cols = ['information','text_list']

    for col in cols:
        print(col)
        df = df.explode(col).reset_index(drop=True)
        df = df.drop(columns=[col]).join(df[col].apply(pd.Series), rsuffix=f".{col}")

    df['content'] = df['content'].apply(lambda x: x[0] if isinstance(x,list) else '')
    df['region_poly'] = df['region_poly'].apply(lambda x: tuple(x)) # lists are mutable 
    df['content_type'] = df['content'].apply(lambda x: type(x).__name__)
    
    # 2
    df = df[df.category_name.apply(lambda x: x in filter_criteria)]
    
    # 3
    aggr_level = ['page', 'region_poly']
    grouped_df = df.groupby(aggr_level)
    # Concatenate the content of the 'content' column with spaces
    df_agg = df.groupby(aggr_level)['content'].apply(lambda x: ' '.join(x)).reset_index()
    
    return df, df_agg
    


In [65]:
df, df_agg = result_to_chunks(final_result)

information
text_list


### ALL TOGETHER NOW!

In [80]:
def check_species_processed(root_path, 
                            processed_list=['Leptonycteris yerbabuenae', 'Leptonycteris nivalis', 'Melipona beecheii']):
    species_subfolders = os.listdir(root_path)
    pending_process_species = list(set(species_subfolders)-set(processed_list))
    return pending_process_species 


In [131]:
def get_list_paths(root_path, 
               input_subfolder_sufix="_bibliografía", 
               output_subfolder = "output"
               ):

    path_dics = []
    
    pending_process_species = check_species_processed(root_path)

    for species_folder in pending_process_species:
        species_folder_path = Path(root_path,species_folder)
        file_name_list = os.listdir(Path(root_path,species_folder,species_folder+'_bibliografía'))
        for file_name in file_name_list:
            path_dics.append(
                dict(
                    file_input_path = Path(species_folder_path,str(species_folder_path.name)+input_subfolder_sufix,file_name),
                    folder_output_path = Path(species_folder_path,output_subfolder),
                    species_folder = species_folder,
                    file_name = file_name
                )
            )
    return path_dics

In [132]:
root_path = '/home/camilo/Documents/by_species'
list_paths = get_list_paths(root_path)
list_paths[0].get('file_input_path')

PosixPath('/home/camilo/Documents/by_species/test species_A/test species_A_bibliografía/Arita, 1991.pdf')

In [114]:
list_paths[0].keys(), \
list_paths[0].get('folder_output_path').parent.name

(dict_keys(['file_input_path', 'folder_output_path', 'species_folder', 'file_name']),
 'test species_A')

In [172]:
import logging

logging.basicConfig(level=logging.DEBUG, filename="py_log.log",filemode="w",
                    format="%(asctime)s %(levelname)s %(message)s")

In [173]:
print(os.getcwd())

/home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain


In [185]:
def loop_OCR(root_path): 
    '''
    Expected folder Structure :

    root_folder/
        species A/
            species A_bibiliografía/
                file_A1.pdf
                file_A2.pdf
            output/
                Doc_A1
                Doc_A2
    
    Note: The subfolders under root_folde define the species-name. 
    '''
    list_paths = get_list_paths(root_path)
    final_results = []

    for file_path_dict in list_paths[0:2]:#[0:1] is just for testing
        pdf_path = file_path_dict.get('file_input_path')
        output_folder = file_path_dict.get('folder_output_path')
        if not os.path.exists(output_folder):
            print("Creating output folder in : {output_folder.parent.name}")
            os.mkdir(output_folder)

        image_list = load_whole_pdf(str(pdf_path))

        try:
            final_result = whole_pdf_conversion_example(image_list)
            
            #final_result = [{**d, "file_path_dict": file_path_dict} for d in final_result]
        
        except:
            final_result = {'error_species':file_path_dict.get('species_folder'), 
                            'error_file': file_path_dict.get('file_input_path')
                            }
            logging.error("Custom_error_msg", exc_info=True)
        
        final_results.append(final_result)
    
    return final_results
    

In [186]:
!pwd

/home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain


In [187]:
final_resuls = loop_OCR(root_path)

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain/modules/../../../DocumentUnderstanding/DocXLayout/../../exp/ctdet_subfield/default
heads {'hm': 11, 'cls': 4, 'ftype': 3, 'wh': 8, 'hm_sub': 2, 'wh_sub': 8, 'reg': 2, 'reg_sub': 2}
Namespace(task='ctdet_subfield', dataset='huntie', test=False, data_src='default', exp_id='default', vis_corner=0, convert_onnx=0, onnx_path='auto', debug=0, load_model='/home/DocXLayout_231012.pth', resume=False, gpus=[0], num_workers=16, not_cuda_benchmark=False, seed=317, print_iter=0, hide_data_time=False, save_all=False, metric='loss', vis_thresh=0.3, nms_thresh=0.3, corner_thresh=0.3, debugger_theme='white', arch='dlav0subfield_34', head_conv=256, down_ratio=4, input_res=768, input_h=768, input_w=768, lr=0.000125, lr_step=[80], NotFixList='', num_epochs=90, batch_size=32, master_batch_size=32, num_iters=-1, val_intervals=5, trainval=False, negative=False, adamW

2024-09-10 17:05:15,687 - modelscope - WARNING - Model revision not specified, use revision: v1.0.1
2024-09-10 17:05:16,202 - modelscope - INFO - initiate model from /home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo
2024-09-10 17:05:16,202 - modelscope - INFO - initiate model from location /home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo.
2024-09-10 17:05:16,205 - modelscope - WARNING - No preprocessor field found in cfg.
2024-09-10 17:05:16,206 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-09-10 17:05:16,207 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo'}. trying to build by task and model information.
2024-09-10 17:05:16,207 - modelscope - WARNING - Find task: ocr-detection, model type: None. Insu

09/10/2024 17:05:18 - INFO - tensorflow -   Restoring parameters from /home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo/tf_ckpts/checkpoint-80000


2024-09-10 17:05:21,519 - modelscope - WARNING - Model revision not specified, use revision: v2.4.0
2024-09-10 17:05:22,111 - modelscope - INFO - initiate model from /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo
2024-09-10 17:05:22,112 - modelscope - INFO - initiate model from location /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo.
2024-09-10 17:05:22,114 - modelscope - INFO - initialize model from /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo
2024-09-10 17:05:23,139 - modelscope - INFO - loading model from dir /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo
2024-09-10 17:05:23,188 - modelscope - INFO - loading model done
2024-09-10 17:05:25.206340: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retu

[{'page': 0, 'information': [{'category_index': 0, 'category_name': 'title', 'region_poly': [79, 194, 986, 194, 986, 265, 79, 265], 'text_list': [{'position': [81, 191, 986, 191, 986, 222, 81, 222], 'content': ['Spatial Segregation in Long-Nosed Bats, Leptonycteris nivalis and']}, {'position': [82, 230, 547, 231, 547, 261, 81, 261], 'content': ['Leptonycteris curasoae, in Mexico']}]}, {'category_index': 0, 'category_name': 'title', 'region_poly': [82, 466, 140, 466, 140, 481, 82, 481], 'text_list': [{'position': [82, 465, 137, 465, 137, 479, 82, 479], 'content': ['1 author:']}]}, {'category_index': 1, 'category_name': 'figure', 'region_poly': [92, 771, 128, 771, 128, 794, 92, 794], 'text_list': []}, {'category_index': 1, 'category_name': 'figure', 'region_poly': [92, 708, 129, 708, 129, 731, 92, 731], 'text_list': []}, {'category_index': 1, 'category_name': 'figure', 'region_poly': [82, 506, 124, 506, 124, 531, 82, 531], 'text_list': []}, {'category_index': 1, 'category_name': 'figure'

2024-09-10 17:07:09,312 - modelscope - WARNING - Model revision not specified, use revision: v1.0.1
2024-09-10 17:07:10,590 - modelscope - INFO - initiate model from /home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo
2024-09-10 17:07:10,592 - modelscope - INFO - initiate model from location /home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo.
2024-09-10 17:07:10,602 - modelscope - WARNING - No preprocessor field found in cfg.
2024-09-10 17:07:10,604 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-09-10 17:07:10,606 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo'}. trying to build by task and model information.
2024-09-10 17:07:10,610 - modelscope - WARNING - Find task: ocr-detection, model type: None. Insu

09/10/2024 17:07:13 - INFO - tensorflow -   Restoring parameters from /home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo/tf_ckpts/checkpoint-80000


2024-09-10 17:07:15,995 - modelscope - WARNING - Model revision not specified, use revision: v2.4.0
2024-09-10 17:07:16,451 - modelscope - INFO - initiate model from /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo
2024-09-10 17:07:16,452 - modelscope - INFO - initiate model from location /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo.
2024-09-10 17:07:16,454 - modelscope - INFO - initialize model from /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo
2024-09-10 17:07:16,762 - modelscope - INFO - loading model from dir /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo
2024-09-10 17:07:16,799 - modelscope - INFO - loading model done
2024-09-10 17:07:18.733342: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retu

09/10/2024 17:07:57 - ERROR - root -   Custom_error_msg
Traceback (most recent call last):
  File "/home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/rapid_latex_ocr/main.py", line 102, in __call__
    resizered_img = self.loop_image_resizer(img)
  File "/home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/rapid_latex_ocr/main.py", line 126, in loop_image_resizer
    input_image = self.pre_pro.minmax_size(pad_img).convert("RGB")
  File "/home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/rapid_latex_ocr/utils.py", line 78, in minmax_size
    img = img.resize(size.astype(int), Image.BILINEAR)
  File "/home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/PIL/Image.py", line 2297, in resize
    if self.size == size and box == (0, 0) + self.size:
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

The above ex

In [115]:
list_paths[0:1]

[{'file_input_path': PosixPath('/home/camilo/Documents/by_species/test species_A/_bibliografía/Arita, 1991.pdf'),
  'folder_output_path': PosixPath('/home/camilo/Documents/by_species/test species_A/output'),
  'species_folder': 'test species_A',
  'file_name': 'Arita, 1991.pdf'}]

In [188]:
final_resuls

[[{'page': 0,
   'information': [{'category_index': 0,
     'category_name': 'title',
     'region_poly': [79, 194, 986, 194, 986, 265, 79, 265],
     'text_list': [{'position': [81, 191, 986, 191, 986, 222, 81, 222],
       'content': ['Spatial Segregation in Long-Nosed Bats, Leptonycteris nivalis and']},
      {'position': [82, 230, 547, 231, 547, 261, 81, 261],
       'content': ['Leptonycteris curasoae, in Mexico']}]},
    {'category_index': 0,
     'category_name': 'title',
     'region_poly': [82, 466, 140, 466, 140, 481, 82, 481],
     'text_list': [{'position': [82, 465, 137, 465, 137, 479, 82, 479],
       'content': ['1 author:']}]},
    {'category_index': 1,
     'category_name': 'figure',
     'region_poly': [92, 771, 128, 771, 128, 794, 92, 794],
     'text_list': []},
    {'category_index': 1,
     'category_name': 'figure',
     'region_poly': [92, 708, 129, 708, 129, 731, 92, 731],
     'text_list': []},
    {'category_index': 1,
     'category_name': 'figure',
     're

In [138]:
cell = {'page': 0, 'information': [{'category_index': 0, 'category_name': 'title', 'region_poly': [79, 194, 986, 194, 986, 265, 79, 265], 'text_list': [{'position': [81, 191, 986, 191, 986, 222, 81, 222], 'content': ['Spatial Segregation in Long-Nosed Bats, Leptonycteris nivalis and']}, {'position': [82, 230, 547, 231, 547, 261, 81, 261], 'content': ['Leptonycteris curasoae, in Mexico']}]}, {'category_index': 0, 'category_name': 'title', 'region_poly': [82, 466, 140, 466, 140, 481, 82, 481], 'text_list': [{'position': [82, 465, 137, 465, 137, 479, 82, 479], 'content': ['1 author:']}]}, {'category_index': 1, 'category_name': 'figure', 'region_poly': [92, 771, 128, 771, 128, 794, 92, 794], 'text_list': []}, {'category_index': 1, 'category_name': 'figure', 'region_poly': [92, 708, 129, 708, 129, 731, 92, 731], 'text_list': []}, {'category_index': 1, 'category_name': 'figure', 'region_poly': [82, 506, 124, 506, 124, 531, 82, 531], 'text_list': []}, {'category_index': 1, 'category_name': 'figure', 'region_poly': [142, 775, 319, 775, 319, 790, 142, 790], 'text_list': [{'position': [142, 774, 193, 774, 193, 788, 142, 787], 'content': ['Doctoral']}]}, {'category_index': 2, 'category_name': 'plain text', 'region_poly': [81, 657, 570, 657, 570, 672, 81, 672], 'text_list': [{'position': [81, 656, 568, 656, 568, 671, 81, 671], 'content': ['Some of the authors ofthis publication are also working on these related projects:']}]}, {'category_index': 2, 'category_name': 'plain text', 'region_poly': [120, 524, 383, 524, 383, 543, 120, 543], 'text_list': [{'position': [136, 525, 384, 523, 384, 538, 136, 539], 'content': ['Universidad Nacional Autonoma de Mexico']}]}, {'category_index': 2, 'category_name': 'plain text', 'region_poly': [132, 550, 316, 550, 316, 566, 133, 566], 'text_list': [{'position': [134, 550, 311, 549, 311, 563, 135, 564], 'content': ['89 PUBLICATIONS 3,591CITATIONS']}]}, {'category_index': 2, 'category_name': 'plain text', 'region_poly': [86, 302, 372, 302, 372, 322, 86, 322], 'text_list': [{'position': [81, 302, 221, 301, 221, 315, 81, 316], 'content': ['Article in Joumal of Ma']}, {'position': [238, 302, 303, 302, 303, 317, 238, 316], 'content': ['nalogy·No']}, {'position': [300, 302, 374, 302, 374, 316, 300, 316], 'content': ['vember 1991']}]}, {'category_index': 2, 'category_name': 'plain text', 'region_poly': [144, 584, 218, 584, 218, 600, 144, 600], 'text_list': [{'position': [150, 584, 212, 583, 212, 596, 151, 596], 'content': ['SEE PROFILE']}]}, {'category_index': 2, 'category_name': 'plain text', 'region_poly': [87, 325, 164, 325, 164, 337, 87, 337], 'text_list': []}, {'category_index': 2, 'category_name': 'plain text', 'region_poly': [136, 504, 223, 504, 223, 520, 136, 520], 'text_list': [{'position': [135, 503, 152, 503, 152, 515, 136, 515], 'content': ['He']}, {'position': [176, 503, 214, 503, 214, 516, 176, 516], 'content': ['TArita']}]}, {'category_index': 2, 'category_name': 'plain text', 'region_poly': [152, 714, 433, 714, 433, 729, 152, 729], 'text_list': [{'position': [141, 713, 393, 713, 393, 727, 141, 727], 'content': ['ecologia y conservacion de carnivoros View']}]}, {'category_index': 2, 'category_name': 'plain text', 'region_poly': [614, 389, 656, 389, 656, 424, 614, 424], 'text_list': [{'position': [622, 392, 650, 392, 650, 404, 622, 404], 'content': ['READS']}, {'position': [617, 411, 643, 411, 643, 425, 617, 425], 'content': ['386']}]}, {'category_index': 2, 'category_name': 'plain text', 'region_poly': [87, 403, 117, 403, 117, 429, 87, 429], 'text_list': [{'position': [81, 411, 97, 411, 97, 425, 81, 425], 'content': ['58']}, {'position': [105, 415, 105, 421, 90, 421, 90, 415], 'content': ['8']}]}, {'category_index': 3, 'category_name': 'header', 'region_poly': [1027, 71, 1177, 71, 1177, 91, 1027, 91], 'text_list': [{'position': [1026, 67, 1177, 68, 1177, 90, 1026, 90], 'content': ['ResearchGate']}]}, {'category_index': 3, 'category_name': 'header', 'region_poly': [82, 149, 694, 149, 694, 166, 82, 166], 'text_list': [{'position': [81, 150, 419, 150, 419, 163, 81, 164], 'content': ['Seediscussions, stats,and author profiles for this publication at:']}, {'position': [694, 151, 694, 162, 681, 162, 681, 151], 'content': ['91']}]}, {'category_index': 6, 'category_name': 'footer', 'region_poly': [89, 1673, 561, 1673, 561, 1690, 89, 1690], 'text_list': [{'position': [88, 1672, 364, 1670, 364, 1685, 88, 1687], 'content': ['All content following this page was uploaded by']}, {'position': [430, 1672, 561, 1671, 561, 1686, 430, 1687], 'content': ['ita on 25 January 2015.']}]}, {'category_index': 6, 'category_name': 'footer', 'region_poly': [91, 1706, 373, 1706, 373, 1722, 91, 1722], 'text_list': [{'position': [88, 1705, 104, 1705, 104, 1717, 88, 1717], 'content': ['The']}, {'position': [115, 1706, 145, 1706, 145, 1719, 115, 1719], 'content': ['serhas']}, {'position': [265, 1705, 316, 1705, 317, 1718, 265, 1719], 'content': ['of the dowr']}, {'position': [341, 1706, 369, 1705, 369, 1718, 341, 1718], 'content': ['ed file.']}]}]}
type(cell)

dict